# Olá

Autor: Felipe Marques de Almeida

<img src="https://logodownload.org/wp-content/uploads/2014/09/twitter-logo-1.png" width="20"> [@fmarquesalmeida](https://twitter.com/fmarquesalmeida)

Olá, este `jupyter notebook` foi criado com o objetivo de providenciar um rápido guia, bem simples, de como executar e utilizar `cadernos jupyter`. Estes cadernos serão utilizados para ministrar a aula prática do minicurso de Bioinformática da XX semabio UnB.

> Este minicurso estará sempre publicamente disponível no github, sinta-se a vontade para utilizá-lo e revisitá-lo quando quiser. 
>
> -- https://fmalmeida.github.io/minicurso_bioinfo_semabio_XX_UnB/

# Aula prática

Na aula prática deste minicurso iremos aprender como funcionam e como utilizar algumas das principais ferramentas e bancos de dados utilizados atualmente na Bioinformática. Veremos alguns dos principais recursos e bancos de dados que estão sendo utilizados para o estudo, monitoramento e combate do SARS-CoV-2.

### Objetivo

    Descobrir e entender como ferramentas de Bioinformática podem auxiliar no controle e combate de epidemias.

# 1. Traçando a origem do novo Coronavirus (2019-nCoV)

> De onde surgiu o 2019-nCoV?

Para tentar responder esta pergunta, precisamos criar uma [árvore filogenética][arvore] do vírus com outros vírus SARS. A árvore nos fornecerá uma representação visual do caminho evolutivo mais provável das espécies. Uma árvore filogenética é construída com base em um arquivo de alinhamento múltiplo de sequências. Estas sequências podem ser genes ou proteínas pré-definidas, ou até mesmo genomas inteiros.

No contexto da era genômica, tem-se a facilidade de sequenciar o genoma completo do vírus de diversos pacientes. Isso permite comparar diversos genomas relativos ao surto para verificar se as conclusões que tiramos com os dois genomas acima se mantém. Devido ao tamanho pequeno dos genomas virais, quase sempre adota-se o alinhamento e filogenia utilizando o genoma completo, a não ser que se deseje traçar o caminho evolutivo de um gene em específico.

Desta forma, se coletarmos outros genomas de surtos anteriores de outros Coronavirus (MERS [middle east respiratory syndrome], SARS [Severe acute respiratory syndrome]) e também genomas de outros hospedeiros (morcegos, camelos), pode-se postular qual o hospedeiro original do 2019-nCoV.

No [NCBI][ncbi] foi feita uma seleção prévia de alguns genomas completos de diversos tipos de Coronavirus, cujos números de acesso são:

> MN908947.3<br>
MG772933.1<br>
NC_004718.3<br>
MN988669.1<br>
MN988668.1<br>
MN994467.1<br>
KX574227.1<br>
MF598691.1<br>
MN985325.1<br>

**Estes números de acesso foram salvos em um arquivo chamado "lista_accs_cov_origem.txt"**

## 1.1 Baixando os genomas virais

O primeiro passo para realizarmos a análise filogenética é, obtermos as sequências genômicas em [arquivos FASTA][fasta]. Para isso utilizaremos o programa [ncbi-acc-download][acc-down].

> Este programa já se encontra pré-instalado no nosso caderno de exercícios. Podemos usar o comando `ncbi-acc-download -h` para checar como utilizá-lo, veja abaixo:

[ncbi]: https://www.ncbi.nlm.nih.gov/
[arvore]: https://pt.khanacademy.org/science/biology/her/tree-of-life/a/phylogenetic-trees
[fasta]: https://pt.wikipedia.org/wiki/Formato_FASTA
[acc-down]: https://github.com/kblin/ncbi-acc-download

In [ ]:
%%bash
ncbi-acc-download -h

---

Ao observar a mensagem de ajuda acima, percebemos que a forma mais comum de utilização do programa é `ncbi-acc-download <NUM-DE-ACESSO>`. Portanto, se levarmos em conta que salvamos todos os números de acesso dos genomas que queremos em um arquivo chamado "lista_accs_cov_origem.txt" podemos utilizá-lo em conjunto com o comando `cat` e `ncbi-acc-download` para baixar todos os genomas de uma vez, ao invés de precisar acessar 1 por 1 no banco de dados

**Linha de comando:**
```bash
# Tente executar esta linha de comando na célula de código abaixo.
ncbi-acc-download -F fasta -o genomas_cov_para_filogenia.fasta $(cat lista_accs_cov_origem.txt)
```

In [1]:
%%bash
# tente criar seu código aqui! Não precisa remover esta linha, basta digitar o código abaixo.

---
No código acima, utilizamos o programa `ncbi-acc-download` para baixar em formato fasta (`-F fasta`) todos os genomas presentes na lista (`$(cat lista_accs_cov_origem.txt)`) e salvá-los em um único arquivo chamado genomas_cov_para_filogenia.fasta.

Execute a linha de comando abaixo para verificar que todos os 9 genomas desejados estão presentes no arquivo final:

In [2]:
%%bash
grep ">" genomas_cov_para_filogenia.fasta

>MN908947.3 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
>MG772933.1 Bat SARS-like coronavirus isolate bat-SL-CoVZC45, complete genome
>NC_004718.3 SARS coronavirus Tor2, complete genome
>MN988669.1 Severe acute respiratory syndrome coronavirus 2 isolate 2019-nCoV WHU02, complete genome
>MN988668.1 Severe acute respiratory syndrome coronavirus 2 isolate 2019-nCoV WHU01, complete genome
>MN994467.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/CA-CDC-CA1/2020, complete genome
>KX574227.1 Bat coronavirus isolate PREDICT/PDF-2180, complete genome
>MF598691.1 Middle East respiratory syndrome-related coronavirus strain camel/UAE_B104_2015, complete genome
>MN985325.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/WA-CDC-WA1/2020, complete genome


## 1.2 Alinhando os genomas

O segundo passo, após a obtenção dos genomas desejados é a realização de um [alinhamento múltiplo de sequências](https://pt.wikipedia.org/wiki/Alinhamento_m%C3%BAltiplo_de_sequ%C3%AAncias). Em termos bem gerais, alinhar sequências consiste em posicionar as sequências "alinhadas" em colunas da melhor forma possível com o objetivo de encontrar regiões de similaridade entre elas, como mostrado na figura abaixo:


<figure>
    <center>
        <img src="https://img.vivaolinux.com.br/imagens/artigos/comunidade/clustalx_sequencia.gif" width="50%">
    </center>
        <figcaption>Fig. 1. Exemplificação de alinhamento múltiplo de sequências. Regiões conservadas, ou seja, regiões de similaridade, são coloridas com a mesma cor de forma a permitir a fácil identificação da conservação entre as sequências. São estas informações de similaridade que são usadas por programas de filogenia para "desenhar" árvores filogenéticas.</figcaption>
</figure>

Um dos principais programas utilizados na atualidade para a realização do alinhamento múltiplo entre sequências é o programa [MAFFT](https://mafft.cbrc.jp/alignment/software/).

> Este programa já se encontra pré-instalado em nosso caderno de exercícios. Podemos usar o comando `mafft -h` para entender sua utilização, que é bastante simples.

In [ ]:
%%bash
mafft -h || exit 0 # o comando "|| exit 0" está aí somente para evitar o código de erro já que o programa não possui uma chamada para a ajuda.

---

Ao observar a mensagem de ajuda acima, percebemos que a forma mais comum de utilização do programa é `mafft <ARQUIVO FASTA> > <ARQUIVO FINAL DE ALINHAMENTO>`. Portanto, podemos utilizar o programa para obter um alinhamento das nossas sequências de coronnavírus da seguinte forma:

**Linha de comando:**
```bash
# Tente executar esta linha de comando na célula de código abaixo.
mafft --auto --clustalout genomas_cov_para_filogenia.fasta > genomas_cov_alinhados.clustal
```

Obs: Nesta linha de comando, o parâmetros `--auto` permite que o programa escolha os melhores parâmetros sozinho e o parâmetro `--clustalout` solicita que o resultado seja escrito em um formato geralmente mais utilizado chamado clustal.

In [3]:
%%bash
# tente criar seu código aqui! Não precisa remover esta linha, basta digitar o código abaixo.
# Lembre-se de não usar o "#" no início das linhas pois ele indica linhas de comentário

nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
    1 / 9
done.

Constructing a UPGMA tree (efffree=0) ... 
    0 / 9
done.

Progressive alignment 1/2... 
STEP     6 / 8 
len1=29642, len2=30123, Switching to the memsave mode
STEP     7 / 8 mDP 00001 / 00084DP 00002 / 00084DP 00003 / 00084DP 00004 / 00084DP 00005 / 00084DP 00006 / 00084DP 00007 / 00084DP 00008 / 00084DP 00009 / 00084DP 00010 / 00084DP 00011 / 00084DP 00012 / 00084DP 00013 / 00084DP 00014 / 00084DP 00015 / 00084DP 00016 / 00084DP 00017 / 00084DP 00018 / 00084DP 00019 / 00084DP 00020 / 00084DP 00021 / 00084DP 00022 / 00084DP 00023 / 00084DP 00024 / 00084DP 00025 / 00084DP 00026 / 00084DP 00027 / 00084DP 00028 / 00084DP 00029 / 00084DP 00030 / 00084DP 00031 / 00084DP 00032 / 00084DP 00033 / 00084DP 00034 / 00084DP 00035 / 00084DP 00036 / 00084DP 00037 / 00084DP 00038 /

---
Após executar o comando acima, podemos utilizar o comando `head` para verificar as 10 primeiras linhas do resultado e ver se tudo está como desejamos:

In [4]:
%%bash
head genomas_cov_alinhados.clustal

CLUSTAL format alignment by MAFFT FFT-NS-2 (v7.480)


MN908947.3      attaaaggtttataccttcccaggtaacaaaccaaccaactttcgatctcttgtagatct
MG772933.1      atattaggtttttaccttcccaggtaacaaaccaactaactctcgatctcttgtagatct
NC_004718.3     atattaggtttttacctacccagg--aaaagccaaccaac-ctcgatctcttgtagatct
MN988669.1      -ttaaaggtttataccttcccaggtaacaaaccaaccaactttcgatctcttgtagatct
MN988668.1      -ttaaaggtttataccttcccaggtaacaaaccaaccaactttcgatctcttgtagatct
MN994467.1      attaaaggtttataccttcccaggtaacaaaccaaccaactttcgatctcttgtagatct
KX574227.1      ------------------------------------------------------------


## 1.3 Construindo a árvore filogenética

Finalmente, após termos alinhado todos os genomas Coronavírus selecionados, podemos utilizar o resultado para a construção de uma árvore filogenética. Existem diversos programas ([lista de programas](https://en.wikipedia.org/wiki/List_of_phylogenetics_software)), tanto em linha de comando quanto em aplicações web que podem ser utilizados para a construção de árvores filogenéticas.

Ao escolher um programa devemos sempre ter em mente que não existe um programa ideal, que resolva todas as necessidades de uma análise. Cada programa possui uma metodologia e algoritmo diferente que lhes proprocionam pontos fracos e fortes que devem sempre ser considerados.

Para o nosso caso, como desejamos obter somente uma prévia da estrutura da árvore filogenética, temos poucas sequências e desejamos priorizar velocidade e pouco consumo de memória, utilizaremos o programa [iqtree](http://www.iqtree.org/).

> Este programa já se encontra pré-instalado em nosso caderno de exercícios. Podemos usar o comando `iqtree -h` para entender sua utilização, que é bastante simples.

In [ ]:
%%bash
iqtree -h

---
Ao observar a mensagem de ajuda acima, percebemos que a forma mais comum de utilização do programa é `iqtree -s <ARQUIVO DE ALINHAMENTO>`.

> **Como poderíamos usar este programa para baixar as sequências de genoma listadas anteriormente?**

Este programa irá produzir diversos arquivos em nossa pasta que são utilizados para o cálcula e predição da árvore filogenética que será armazenada em um arquivo final com o sufixo `.treefile`.

In [ ]:
%%bash
# tente criar seu código aqui! Não precisa remover esta linha, basta digitar o código abaixo.
# Lembre-se de não usar o "#" no início das linhas pois ele indica linhas de comentário

In [ ]:
%%bash
# veja os resultados com o programa "ls"
ls

## 1.4 Visualização da árvore filogenética

Assim como as ferramentas, ou programas, para realizar o alinhamento de sequências e o cálculo da árvore, existem inúmeros programas para a visualização de árvores filogenéticas, em linha de comando ou em aplicações web (veja uma [breve lista de programas](https://www.biostars.org/p/2438/)). Desta forma, assim como na etapa anterior, estamos somente desejando uma visualização rápida dos resultados, focando na velocidade e simplicidade. Tendo isto em mente, utilizaremos para nossa tarefa a ferramenta [plottree](https://github.com/iBiology/plottree).

### 1.4.1 plottree

Para termos uma figura da árvore filogenética de maneira bem simples e rápida temos o programa plottree, que apesar de rápido e eficiente, não permite a edição interativa das árvores geradas.

> Tente executar a seguinte linha de comando:

In [ ]:
%%bash
plottree genomas_cov_alinhados.clustal.treefile -o plottree_fig.png

---
Depois de executar o comando escreva o seguinte código (de \<figure> até \</figure>) na _célula markdown_ em branco abaixo:

```html
<figure>
    <center>
        <img src="plottree_fig.png" width="30%">
    </center>
    <figcaption>Árvore filogenética desenhada com plottree</figcaption>
</figure>
```

## 2. Calculando / detectando variações

Uma outra tarefa frequente em Bioinformática é a detecção de variações importantes entre genomas de uma espécie. Em um contexto bem geral, estas variações (mutações) podem ser neutras, negativas ou positivas. Geralmente, a frequência dos tipos de mutação é espécie dependente. Quando um indivíduo, ou pequena população de uma espécie acumula um certo nível de variações "não neutras" entre elas, dizemos que esta é uma nova [linhagem](http://www.genomahcov.fiocruz.br/linhagens/).

Atualmente, sabemos que existem 4 principais variantes / linhagens do SARS-CoV-2. São elas:

* A linhagem Alpha (B.1.1.7);
* A linhagem Beta (B.1.351);
* A linhagem Gamma (P.1);
* A linhagem Delta (B.1.617.2);

Dentre elas, atualmente, tem-se bastante preocupação acerca da variante Delta que aparenta ser mais contagiosa e mais letal. Mas, quais seriam, e onde ocorrem, as mutações desta linhagem que culminaram neste [fenótipo](https://pt.wikipedia.org/wiki/Fen%C3%B3tipo) mais preocupante?

De acordo com o [relatório da iniciativa PANGO lineages sobre a linhagem B.1.617.2](https://cov-lineages.org/global_report_B.1.617.2.html) e do [grande consórcio GISAID + Nextstrain](https://covariants.org/variants/21A.S.154K), esta linhagem teve seu primeiro relato na India, em 21 de Novembro de 2020 e, atualmente, já se encontra em 64 países, com mais de 28 mil registros. Segundo aos relatórios de vigilância genômica registrados pelos grupos [PANGO lineages](https://cov-lineages.org/index.html), [Nextstrain](https://nextstrain.org) e [GISAID](https://www.gisaid.org/), existem 12 principais mutações existentes que definem, ou distinguem estas linhagens das outras:

* S:T19R
* S:L452R
* S:T478K
* S:P681R
* S:D950N
* ORF3a:S26L
* M:I82T
* ORF7a:V82A
* ORF7a:T120I
* N:D63G
* N:R203M
* N:D377Y

> Os nomes (ou letras) antes do ":" indica a proteína na qual aquela mutação é encontrada. Vale lembrar, que estas mutações são mutações encontradas na linhagem Delta, mas não significa dizer que todo vírus da linhagem terá todas as mutações. Elas têm frequências diferentes. Com estas mutações, as ferramentas categorizam como pertecente a mesma linhagem quando pelo menos 5 destas mutações estão presentes.

> Dado um genoma qualquer da linhagem Delta, vamos tentar identificar estas variações em seu genoma?

### 2.1. Realizando a análise

> Como exemplo, vamos utilizar um isolado de Maio de 2021, na Índia (MZ310580.1). [Clique aqui para baixar o genoma](sars-cov-2-example-genome.fasta).

Apesar de podermos realizar estas análises todas pela linha de comando, acredito que o minicurso tomaria um percurso muito específico e talvez se tornasse bastante difícil de ser acompanhado. Desta forma, explicarei como usufruir das ferramentas e bancos de dados online (gratuitos) disponíveis de forma a destrinchar e melhor entender como são feitas análises de vigilância envolvendo o SARS-CoV-2.

> Por exemplo, dado o sequenciamento do genoma de um novo isolado, como podemos definir suas linhagens? Suas mutações? Seu posicionamento na árvore filogenética? Como saber monitorar as linhagens e mutações no mundo? Etc.

#### 2.1.1. Identificando a linhagem

Para identificação da linhagem, podemos rapidamente usar o programa [pangolin](https://pangolin.cog-uk.io/) através da sua interface web ou de linha de comando.

> Execute os códigos abaixo e veja os resultados, você consegue me dizer qual a linhagem do genoma?

In [ ]:
%%bash
# use pangolin para definir a linhagem
pangolin -o pango_results sars-cov-2-example-genome.fasta

In [ ]:
# veja os resultados
import pandas as pd
pd.read_csv('pango_results/lineage_report.csv', sep=',')

#### 2.1.2. Como identificar as mutações?

O programa [Nextclade](https://clades.nextstrain.org/), possui uma ferramenta em linha de comando capaz de rapidamente identificar as mutações e identificar o clado de um genoma em questão.

> Execute os códigos abaixo e veja os resultados. Quantas mutações este genoma tem? Elas são referentes a linhagem Delta? Qual a proteína / gene com mais mutações?

In [ ]:
%%bash
# executando nextclade
nextclade -i sars-cov-2-example-genome.fasta -t nextclade_results.tsv

In [ ]:
# veja os resultados
import pandas as pd
pd.read_csv('nextclade_results.tsv', sep='\t')

#### 2.1.3 Usando as aplicações web

Apesar de também possuírem linhas de comando, a maioria, o quase todas, as ferramentas utilizadas para as análises de SARS-CoV-2 possuem aplicações web. Estas aplicações web oferecem as seguintes vantagens:

1. Atualização constante de seus banco de dados
2. Integração rápida dos resultados com comparações com sequências de bancos de dados
3. Permite a análise rápido e fácil de _N_ genomas ao mesmo tempo
4. É mais simples para pessoas não familiarizadas com linha de comando, o que facilita e incentiva sua utilização por qualquer unidade de saúde de modo a possuir uma melhor vigilância do vírus e permitir uma padronização das análises
5. Permite a existência de aplicações "mais" específicas que utilizem seus dados para a vigilância genômica de cepas e mutações

Neste contexto, deixo abaixo uma lista dos principais recursos voltados ao acompanhamento e combate do SARS-CoV-2:

> Abordaremos alguns em aula, os outros ficam de leitura complementar.

1. [Pangolin](https://pangolin.cog-uk.io/), utilizando para identificar linhagens
2. [Nextclade](https://clades.nextstrain.org/), utilizado para classificar sequências, identificar mutações e filogenia
3. [covSPECTRUM](https://cov-spectrum.ethz.ch/explore/Switzerland/AllSamples/AllTimes), permite a vigilância e observação de estatísticas sobre variantes conhecidas em cada país
4. [REGENERON](https://covid19dashboard.regeneron.com/?tab=Home&subTab=Europe), permite a investigação das estatísticas e sequências da pandemia em um contexto global
5. [Mutation Reports](https://outbreak.info/situation-reports), possui relatórios simples e interessantes sobre as mutações de cada linhagem
6. [CoVariants](https://covariants.org/), possui relatórios um pouco mais completos, sobre as características de cada variante
7. [Covid-Miner](https://covid-miner.ifo.gov.it/app/home), uma aplicação especializada na "mineração" de informação sobre covid que permite a investigação de informações relacionadas a variantes como: mutações conhecidas, distribuição georgráfica, etc.
8. [CoVizu](http://filogeneti.ca/covizu/), uma aplicação que permite visualizar a diversidade global do vírus SARS-CoV-2
9. [Covid CG](https://covidcg.org/?tab=location), possibilita a investigação das variações genéticas e dos alelos relacionados as mutações observadas na sequência viral, permitindo se informar sobre a distribuição geográfica e frequência de cada variação genética
10. [Cov-GLUE](http://cov-glue.cvr.gla.ac.uk/#/home), é um programa desenvolvido em conjunto com a Universidade de Glasgow para o monitoramento do vírus no Reino Unido
11. [Mutation Tracker](https://users.math.msu.edu/users/weig/SARS-CoV-2_Mutation_Tracker.html), permite observar o número total de mutações observadas em cada gene do genoma viral
12. [Nextstrain](https://nextstrain.org/ncov/global), uma ferramenta incrível que permite o acompanhamento epidemiológico global completo com informações de tempo, localização, filogenia e variações genéticas, importantíssimo para o entendimento e vigilância de epidemias